La je vais essayer de traduire larticle en un petit algorithme (ou plutot deux un pour voir le deroulement dune partie un pour faire tourner x parties et juste voir les stats) Le programme demande dabord le nombre de joueurs et le mode de jeu deterministe ou stochastique. Ensuite, il genere un graphe dirige aleatoire en fonction du nombre de joueurs, il assigne à chaque joueur une strategie de transfert un voisin unique(deterministe) ou une distribution de probabilites(stochastique), et calcule à la fin les couts à long terme de chaque joueur. Il indique aussi si la configuration obtenue constitue un equilibre de Nash ou non.
Lobjectif principal est de pouvoir faire des tests facilement : soit on lance lalgorithme une seule fois pour observer le déroulement detaillé dune partie, soit on lexecute x fois pour obtenir des statistiques sur la frequence des equilibres de Nash dans des graphes et strategies aleatoires.

In [6]:
from sys import flags
import random
import networkx as nx
import numpy as np
import pandas as pd

class BuckPassingGame:
  def __init__(self,nb,mode,mu=None):
    self.nb=nb
    self.mode=mode
    self.players = [f"J{i+1}" for i in range(nb)]
    self.graph={}
    self.strat={}
    if mu is None :
      self.mu={i:1/nb for i in self.players}

  def Voisinage(self):
    """Créer le voisinnage pour chaque joueur --> random

       Int : self
       Out : None
    """

    for i in self.players:
      Voisins = [n for n in self.players if n!=i]
      Nombre_Voisins = random.randint(1,len(Voisins)) # ici on pourrait limiter ca a par exemple max 1 a x<(n-1) voisins et pas de 1 a (n-1)
      self.graph[i]=random.sample(Voisins,Nombre_Voisins)


  def strategies(self):
    """Créer les stratégies de chaque joueur en différenciant le cas deterministe et stochatsique

       Int : self
       Out : None
    """
    for i in self.players:
      Voisins = self.graph[i]
      if self.mode=="d":
        choix=random.choice(Voisins) # On fait le random.choice qu'une seule fois (car si cresy fait dans la création du dict c'est pas cohérent car on random a chaque itération du dict)
        self.strat[i]={n:1.0 if n==choix else 0.0 for n in Voisins}
      else:
        #self.mode=="s"
        r=[random.random() for n in Voisins]
        s=sum(r)
        self.strat[i] = {n: rv / s for n, rv in zip(Voisins, r)}#cest pour literation en parallele


  def costs_exact(self) :
    """Calcule et revoie le cout pour chaque joueur dans le cas deterministe et stochastique.

       In : self
       Out : le cout pour chaque joueur (dict[str, float])"""

    if self.mode == "d":
      G_func = nx.DiGraph()
      for i in self.players:
        for j,proba in self.strat[i].items():
          if proba==1:
            G_func.add_edge(i,j);

      costs = {p: 0.0 for p in self.players}
      # weakly_connected_components trouve les composantes connexes en ignorant le sens des arêtes


      for component in nx.weakly_connected_components(G_func):
        sousgraphe = G_func.subgraph(component)
        cycles = list(nx.simple_cycles(sousgraphe))
        if cycles:
          cycle=cycles[0]
          A_l=set()
          for noeud in component:
            if nx.has_path(G_func,noeud,cycle[0]): # je sais pas si c'est obligatoire de faire ca --> car tt les noueuds du componant ammene au cycle vu que le ss graphe est un unicycle
              A_l.add(noeud)
          mu_l = sum(self.mu[j] for j in A_l)
          for i in cycle:
            costs[i] = mu_l / len(cycle) #ici j'ai add le /len(cycle) car il faut diviser le cout par tous la taille du cycle (voir 2.6)

      return costs

    else:
            # MODE STOCHASTIQUE
            # Calcul basé sur l'équation (3.6) de l'article : c_i = sum(mu^l * rho^l)
            # Cela revient à calculer la distribution limite de la chaîne de Markov.

            n = len(self.players)
            P = np.zeros((n, n))

            # 1. Construction de la Matrice de Transition P
            # P[i, j] est la probabilité que i passe le problème à j
            for i, p_str in enumerate(self.players):
                strat = self.strat[p_str]
                for j, q_str in enumerate(self.players):
                    P[i, j] = strat.get(q_str, 0.0)

            # 2. Calcul de la matrice limite (P à la puissance "infini")
            # On élève P à une grande puissance pour obtenir le comportement à long terme.
            # Cela gère automatiquement les multiples classes récurrentes et bassins d'attraction.
            # suffisant pour converger numériquement.
            P_limit = np.linalg.matrix_power(P, 2000)

            # 3. Application de la distribution initiale (Nature)
            # Coût final = (Distribution Initiale) * (Matrice Limite)
            mu_vec = np.array([self.mu[p] for p in self.players])

            # Produit vectoriel : mu x P_limit
            couts_vec = np.dot(mu_vec, P_limit)

            return {self.players[i]: couts_vec[i] for i in range(n)}


  def est_equilibre_nash(self, tol=1e-6):
      """
      Vérifie si le profil de stratégies actuel est un Équilibre de Nash (EN).

      Idée :
      - Je calcule d'abord le coût actuel de chaque joueur (profil de base).
      - Puis, pour chaque joueur, je teste toutes ses déviations possibles (changer de voisin).
      - Si au moins une déviation diminue le coût du joueur (au-delà d'une tolérance), alors
        ce n'est pas un EN.

      - En mode déterministe, c'est un test complet (toutes les stratégies possibles sont pures).
      - En mode stochastique, ici je teste seulement des déviations vers stratégies pures (approximation)
      """
      couts_base = self.costs_exact()
      resultats = []

      for joueur in self.players:
          strat_orig = self.strat[joueur].copy()
          voisins = self.graph[joueur]

          for dev in voisins:
              # Déviation vers stratégie pure sur dev
              self.strat[joueur] = {n: 1.0 if n == dev else 0.0 for n in voisins}
              couts_dev = self.costs_exact()
              bool_improvement = couts_dev[joueur] < couts_base[joueur] - tol

              #on stock pour un affichage clair
              resultats.append({
                  "joueur": joueur,
                  "deviation_vers": dev,
                  "new_cost": couts_dev[joueur],
                  "basic_cost": couts_base[joueur],
                  "improve": "yes" if bool_improvement else "no"
              })

              if bool_improvement:
                  self.strat[joueur] = strat_orig  # reset
                  df_resultats = pd.DataFrame(resultats) #clean version
                  return False, df_resultats

          self.strat[joueur] = strat_orig
      df_resultats = pd.DataFrame(resultats) #clean version
      return True, df_resultats



  def simulation(self, T=100000):
      """Approximation par simulation longue
      Idée :
      - Je simule T étapes de passage du buck.
      - Je compte combien de fois chaque joueur reçoit le buck.
      - J’approxime le coût par fréquence empirique : compteur[i] / T.
      """
      compteur = {p: 0 for p in self.players}
      detenteur = random.choices(self.players, weights=list(self.mu.values()))[0]
      for _ in range(T):
          compteur[detenteur] += 1
          strat = self.strat[detenteur]
          detenteur = random.choices(list(strat.keys()), weights=list(strat.values()))[0]
      return {p: compteur[p] / T for p in self.players}




  def passer_le_probleme(self, nb_etapes=20, calcul_couts=True):
      """Simulation du jeu sans batch"""

      detenteur = random.choices(self.players, weights=list(self.mu.values()))[0]
      print(f">>> DÉBUT DU JEU : {detenteur} reçoit le problème !\n")

      for etape in range(1, nb_etapes + 1):
          print(f"--- Étape {etape} ---")
          print(f"Le problème est chez {detenteur}.")
          strat = self.strat[detenteur]
          voisins = list(strat.keys())
          probs = list(strat.values())

          #cas deterministe
          if self.mode == "d":
              print("voisins : " + str(voisins) + "\nprobs : " + str(probs))
              cible = voisins[probs.index(1.0)]
              print(f"  {detenteur} (Déterministe) → passe toujours à {cible}")

          #cas stochastique
          else:
              cible = random.choices(voisins, weights=probs)[0]
              p = strat[cible]
              print(f"  {detenteur} (Stochastique) → probs { {k: round(v,2) for k,v in strat.items()} }")
              print(f"  → Tire {cible} (prob {round(p,2)})")
          print(f"  >>> {detenteur} PASSE À {cible} !\n")
          detenteur = cible

      #calcul des couts
      if calcul_couts:
          print("\n--- Coûts à long terme (exact) ---")
          couts = self.costs_exact()
          for j, c in sorted(couts.items(), key=lambda x: self.players.index(x[0])):
              print(f"  {j} : {round(c, 6):.6f} ")

          print(f"Somme des coûts : {round(sum(couts.values()), 6):.6f}")

          #affichage des voisins de chacun
          print("\nAvec pour chaque joueur les voisins et leurs probabilitées :")
          for j, s in sorted(self.strat.items(), key=lambda x: self.players.index(x[0])):
              print(f"  {j} : {s}")

          #verification d'un potentiel EN
          result_EN, df_detail = self.est_equilibre_nash()
          print(f"\nÉquilibre de Nash ? {'OUI' if result_EN else 'NON'}")
          print(f"\n\nDétails des itérations pour la vérifiaction d'un potentiel EN :")
          print(df_detail.to_string(index=False))
          print("\n\n----------------------------------------\n")


  def passer_le_probleme_batch(self, nb_etapes=20, nb_simulations=100):
      """Simulation du jeu avec batch.
         Fait tourner le jeu plusieurs fois et donne des statistiques."""
      nash_count = 0
      couts_cumules = {p: 0.0 for p in self.players}

      for sim in range(1, nb_simulations + 1):
        self.strategies()  # ← AJOUTÉ : génère de nouvelles stratégies à chaque simulation
        detenteur = random.choices(self.players, weights=list(self.mu.values()))[0]
        for etape in range(nb_etapes):
            strat = self.strat[detenteur]
            voisins = list(strat.keys())
            if self.mode == "d":
                detenteur = [v for v, prob in strat.items() if prob == 1.0][0]  # version simplifiée
            else:
                detenteur = random.choices(voisins, weights=list(strat.values()))[0]

        # Calcul des coûts exacts après chaque simulation
        couts = self.costs_exact()
        for p in self.players:
            couts_cumules[p] += couts[p]

        #verification d'un potentiel EN
        result_EN, _ = self.est_equilibre_nash()
        if result_EN:
            nash_count += 1

        #update de la proression
        if sim % max(1, nb_simulations // 10) == 0 or sim == nb_simulations:
            print(f"Simulation {sim}/{nb_simulations} terminée → Équilibres trouvés : {nash_count} ({nash_count/sim*100:.1f}%)")


      #resultat finaux (renvoie en moyenne)
      print("\n=== STATISTIQUES GLOBALES ===")
      print("Coûts moyens par joueur :")
      for p in self.players:
          print(f"  {p} : {couts_cumules[p]/nb_simulations:.6f}")
      print(f"Équilibre de Nash trouvé dans {nash_count}/{nb_simulations} simulations ({nash_count/nb_simulations*100:.2f}%)")
      print("============================\n")


  def path_improvement(self, tol=1e-6, affichage="n", cpt=0):
    """
    Implémente un chemin d’amélioration (improvement path) pour le jeu déterministe.

    Idée de l'article :
    - Tant qu'on n'est pas à l'EN, il existe au moins un joueur avec une déviation profitable.
    - On applique une déviation profitable → on avance d'une étape.
    - La fonction potentiel Ψ diminue strictement à chaque déviation profitable.
    - Donc on finit toujours par converger vers un EN (FIP).

    Ici :
    - cpt compte le nombre d'étapes effectuées.
    - On s'arrête quand est_equilibre_nash() retourne True.
    """

    if self.mode == "d":
      result_EN, df_detail = self.est_equilibre_nash()

      #si oui --> affichage détaillé via un data frame
      if affichage == "y":
        print("\n\n" + df_detail.to_string(index=False))

      if result_EN == True :
        print("\nPas de chemin d'amélioration possible --> EN")

        #Théorème 2.10
        n=len(self.players)
        borne=(n**2)/4 - 1
        print(f"Nombre d'étapes effectuées : {cpt}")
        print(f"Borne théorique max (1/4 * n^2 - 1) : {borne}")
        if cpt <= borne :
          print("Le Théorème 2.10 est respecté.")
        else:
          print("Le Théorème 2.10 n'est pas respecté.")

        return cpt

      else :
        couts_base = self.costs_exact()
        ancien_psi = self.potentiel()

        for joueur in self.players:
            strat_orig = self.strat[joueur].copy()
            voisins = self.graph[joueur]

            for dev in voisins:
                # déviation vers stratégie pure sur dev
                self.strat[joueur] = {n: 1.0 if n == dev else 0.0 for n in voisins}
                couts_dev = self.costs_exact()

                if couts_dev[joueur] < couts_base[joueur] - tol:

                    #on affiche le changement
                    print("\nAncienne stratégie : " + joueur + " : " + str(strat_orig))
                    print("Nouvelle stratégie : " + joueur + " : " + str(self.strat[joueur]))
                    new_psi = self.potentiel()
                    print(f"New potentiel psi ({new_psi}) < ancien potentiel psi ({ancien_psi}) : {new_psi < ancien_psi}")

                    return self.path_improvement(tol, affichage, cpt+1)

            self.strat[joueur] = strat_orig




  def potentiel(self):
    """
    Calcule la fonction potentiel Ψ(s) du jeu déterministe.

    Dans l'article (Théorème 2.8) :
      Ψ(s) = somme_{cycles} (n - taille_du_cycle)

    Ici :
    - Je reconstruis le graphe induit par les choix déterministes.
    - Je trouve les cycles dans chaque composante.
    - Je somme (n - |C|) pour obtenir Ψ(s).

    Intuition :
    - cycles courts → Ψ grand (mauvais)
    - cycles longs → Ψ plus petit (meilleur)
    - casser/allonger les cycles réduit Ψ, donc on converge.
    """
    G = nx.DiGraph()
    for i in self.players:
        for j, p in self.strat[i].items():
            if p == 1:
                G.add_edge(i, j)

    psi = 0
    n = len(self.players)
    for component in nx.weakly_connected_components(G):
        sub = G.subgraph(component)
        cycles = list(nx.simple_cycles(sub))
        if cycles:
            C = cycles[0]
            psi += n - len(C)
    return psi



# POUR FAIRE UN SEUL TEST

In [7]:
#main 1
while True:
    try:
        nb = int(input("Nombre de joueurs (ex: 5) : "))
        if nb > 1:
            break
        print("Doit être au moins 2 joueurs.")
    except:
        print("Entrez un nombre valide.")

while True:
    mode = input("Mode (d = déterministe, s = stochastique) : ").lower()
    if mode in ("d", "s"):
        break
    print("Tapez 'd' ou 's'.")

game = BuckPassingGame(nb, mode)
game.Voisinage()
game.strategies()
game.passer_le_probleme(nb_etapes=20)

Nombre de joueurs (ex: 5) : 3
Mode (d = déterministe, s = stochastique) : d
>>> DÉBUT DU JEU : J3 reçoit le problème !

--- Étape 1 ---
Le problème est chez J3.
voisins : ['J2', 'J1']
probs : [0.0, 1.0]
  J3 (Déterministe) → passe toujours à J1
  >>> J3 PASSE À J1 !

--- Étape 2 ---
Le problème est chez J1.
voisins : ['J3', 'J2']
probs : [0.0, 1.0]
  J1 (Déterministe) → passe toujours à J2
  >>> J1 PASSE À J2 !

--- Étape 3 ---
Le problème est chez J2.
voisins : ['J1', 'J3']
probs : [0.0, 1.0]
  J2 (Déterministe) → passe toujours à J3
  >>> J2 PASSE À J3 !

--- Étape 4 ---
Le problème est chez J3.
voisins : ['J2', 'J1']
probs : [0.0, 1.0]
  J3 (Déterministe) → passe toujours à J1
  >>> J3 PASSE À J1 !

--- Étape 5 ---
Le problème est chez J1.
voisins : ['J3', 'J2']
probs : [0.0, 1.0]
  J1 (Déterministe) → passe toujours à J2
  >>> J1 PASSE À J2 !

--- Étape 6 ---
Le problème est chez J2.
voisins : ['J1', 'J3']
probs : [0.0, 1.0]
  J2 (Déterministe) → passe toujours à J3
  >>> J2 PASSE 

d# POUR FAIRE X TEST

In [ ]:

#main 2
while True:
    try:
        nb = int(input("Nombre de joueurs (ex: 5) : "))
        if nb > 1:
            break
        print("Doit être au moins 2 joueurs.")
    except:
        print("Entrez un nombre valide.")

while True:
    mode = input("Mode (d = déterministe, s = stochastique) : ").lower()
    if mode in ("d", "s"):
        break
    print("Tapez 'd' ou 's'.")

game = BuckPassingGame(nb, mode)
game.Voisinage()
game.strategies()
game.passer_le_probleme_batch(nb_etapes=20, nb_simulations=100)



# Path Improvment (jeu deteministe)

In [ ]:
#On peut coder ici l'improvment path pour le jeu deterministe et montrer via simulation que c'est borné par (1/4)n^2 -1


#main 1
while True:
    try:
        nb = int(input("Nombre de joueurs (ex: 5) : "))
        if nb > 1:
            break
        print("Doit être au moins 2 joueurs.")
    except:
        print("Entrez un nombre valide.")

while True:
    aff = input("Afficher chaque path improvement (y/n) :").lower()
    if aff in ("y", "n"):
        break
    print("Tapez 'y' ou 'n'.")


game = BuckPassingGame(nb, "d")
game.Voisinage()
game.strategies()

print("\n--- Coûts à long terme AVANT PATH IMPROVEMENT (exact) ---")
couts = game.costs_exact()
for j, c in sorted(couts.items(), key=lambda x: game.players.index(x[0])):
    print(f"  {j} : {round(c, 6):.6f} ")

print(f"Somme des coûts : {round(sum(couts.values()), 6):.6f}")


result = game.path_improvement(tol= 1e-6, affichage=aff)

print("\n--- Coûts à long terme APRES PATH IMPROVEMENT (exact) ---")
couts = game.costs_exact()
for j, c in sorted(couts.items(), key=lambda x: game.players.index(x[0])):
    print(f"  {j} : {round(c, 6):.6f} ")

print(f"Somme des coûts : {round(sum(couts.values()), 6):.6f}")

print(f"\n\nNombre de path improvement : {result}")

nb = len(game.players)
print(f"\n\nIs {result} < (1/4)n^2 -1 -->  {result < (1/4)*nb**2 -1} ((1/4)n^2 -1 = {((nb**2/4)) -1})")

Nombre de joueurs (ex: 5) : 10
Afficher chaque path improvement (y/n) :y

--- Coûts à long terme AVANT PATH IMPROVEMENT (exact) ---
  J1 : 0.000000 
  J2 : 0.000000 
  J3 : 0.000000 
  J4 : 0.000000 
  J5 : 0.000000 
  J6 : 0.500000 
  J7 : 0.000000 
  J8 : 0.000000 
  J9 : 0.000000 
  J10 : 0.500000 
Somme des coûts : 1.000000


joueur deviation_vers  new_cost  basic_cost improve
    J1            J10  0.000000         0.0      no
    J1             J6  0.000000         0.0      no
    J1             J4  0.233333         0.0      no
    J1             J7  0.350000         0.0      no
    J1             J2  0.175000         0.0      no
    J1             J3  0.140000         0.0      no
    J1             J9  0.116667         0.0      no
    J1             J5  0.350000         0.0      no
    J2             J6  0.000000         0.0      no
    J2             J8  0.000000         0.0      no
    J2             J4  0.000000         0.0      no
    J2             J7  0.000000         0.0 

# Simulation avec création du graphe manuelle

In [ ]:
while True:
    try:
        nb = int(input("Nombre de joueurs (ex: 3) : "))
        if nb > 1:
            break
        print("Doit être au moins 2 joueurs.")
    except ValueError:
        print("Entrez un nombre valide.")

while True:
    mode = input("Mode (d = déterministe, s = stochastique) : ").lower()
    if mode in ("d", "s"):
        break
    print("Tapez 'd' ou 's'.")

game = BuckPassingGame(nb, mode)

# création du voisinage
print("\n--- Création du voisinage du Graphe ---")
manual_graph = {}
for player in game.players:
    while True:
        neighbors_input = input(f"Entrez les voisins de {player} (séparés par des virgules, ex: J1,J2) : ").replace(" ", "")

        neighbors_list = [n.strip().upper() for n in neighbors_input.split(',')]
        flag_neighbors = True
        final_neighbors = []

        if not neighbors_list:
            print(f"Au moins un voisin doit être spécifié pour {player}.")
            flag_neighbors = False
            break

        for n in neighbors_list:
            if n not in game.players or n == player:
                print(f"Voisin '{n}' invalide pour {player}. Assurez-vous qu'il s'agit d'un autre joueur existant.")
                flag_neighbors = False
                break
            elif n in final_neighbors:
                print(f"Voisin '{n}' dupliqué pour {player}. Veuillez entrer des voisins uniques.")
                flag_neighbors = False
                break
            final_neighbors.append(n)

        if flag_neighbors:
            manual_graph[player] = final_neighbors
            break
game.graph = manual_graph
print("Graphe défini manuellement:", game.graph)


# def des stratégies
print("\n--- Définition des Stratégies ---")
manual_strat = {}
for player in game.players:
    print(f"\nDéfinir la stratégie pour {player} (Voisins possibles: {game.graph.get(player, [])}):")
    neighbors = game.graph.get(player, [])

    if game.mode == "d": # mode deterministe
        while True:
            choice = input(f"  {player} passe à (choisissez un voisin parmi {', '.join(neighbors)}): ").strip().upper()
            if choice in neighbors:
                manual_strat[player] = {n: 1.0 if n == choice else 0.0 for n in neighbors}
                break
            else:
                print("Choix invalide. Le joueur doit passer à un de ses voisins.")
    else: # mode stochastique
        while True:
            probs = {}
            total_prob = 0.0
            print(f"  Entrez les probabilités pour les voisins de {player}:")
            for neighbor in neighbors:
                while True:
                    try:
                        prob_str = input(f"    Probabilité pour {neighbor}: ")
                        prob = float(prob_str)
                        if 0 <= prob <= 1:
                            probs[neighbor] = prob
                            total_prob += prob
                            break
                        else:
                            print("La probabilité doit être entre 0 et 1.")
                    except ValueError:
                        print("Entrée invalide. Veuillez entrer un nombre.")

            if abs(total_prob - 1.0) < 1e-6:
                manual_strat[player] = probs
                break
            else:
                print(f"La somme des probabilités est {total_prob:.2f}, elle doit être de 1.0. Réessayez.")
game.strat = manual_strat
print("Stratégies définies manuellement:", game.strat)

game.passer_le_probleme(nb_etapes=20)

Nombre de joueurs (ex: 3) : 3
Mode (d = déterministe, s = stochastique) : d

--- Création du voisinage du Graphe ---
Entrez les voisins de J1 (séparés par des virgules, ex: J1,J2) : J2,J3
Entrez les voisins de J2 (séparés par des virgules, ex: J1,J2) : J1,J3
Entrez les voisins de J3 (séparés par des virgules, ex: J1,J2) : J2,J1
Graphe défini manuellement: {'J1': ['J2', 'J3'], 'J2': ['J1', 'J3'], 'J3': ['J2', 'J1']}

--- Définition des Stratégies ---

Définir la stratégie pour J1 (Voisins possibles: ['J2', 'J3']):
  J1 passe à (choisissez un voisin parmi J2, J3): J2

Définir la stratégie pour J2 (Voisins possibles: ['J1', 'J3']):
  J2 passe à (choisissez un voisin parmi J1, J3): J1

Définir la stratégie pour J3 (Voisins possibles: ['J2', 'J1']):
  J3 passe à (choisissez un voisin parmi J2, J1): J1
Stratégies définies manuellement: {'J1': {'J2': 1.0, 'J3': 0.0}, 'J2': {'J1': 1.0, 'J3': 0.0}, 'J3': {'J2': 0.0, 'J1': 1.0}}
>>> DÉBUT DU JEU : J2 reçoit le problème !

--- Étape 1 ---
Le prob